In [1]:
#Chargment de différentes bibliothèque
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [2]:
# chargement local des moteurs avec le plug-in.
HTML('''
<script src="./sigma.min.js"></script>
<script src="./sigma-add-method-neighbors.js"></script>

''')

In [3]:
#SCIENCE DES RÉSEAUX.
#Créons une connexion réseau aléatoire avec des nœuds

In [4]:
random.seed(42)

n_nodes = 20
n_edges = 100

graph_data = { 'nodes': [], 'edges': [] }

for i in range(n_nodes):
    graph_data['nodes'].append({
            "id": "n" + str(i),
            "label": "n" + str(i),
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": random.uniform(0.2,1)
        })

for j in range(n_edges):
    x_center = random.uniform(0,1)
    y_center = random.uniform(0,1)
    x_dist = random.uniform(0.1,0.5)
    y_dist = random.uniform(0.2,0.5)
    neighborhood = []
    for node in graph_data['nodes']:
        if abs(node['x'] - x_center) < x_dist:
            if abs(node['y'] - y_center) < y_dist:
                neighborhood.append(int(node['id'].replace('n','')))
    if len(neighborhood) >= 2:
        ends = random.sample(neighborhood,2)
        graph_data['edges'].append({
                "id": "e" + str(j),
                "source": "n" + str(ends[0]),
                "target": "n" + str(ends[1])
            })

In [5]:
pd.DataFrame(graph_data['nodes'])

,id,label,x,y,size
0,n0,n0,0.639427,0.025011,0.420023
1,n1,n1,0.223211,0.736471,0.741360
2,n2,n2,0.892180,0.086939,0.537537
3,n3,n3,0.029797,0.218638,0.604284
4,n4,n4,0.026536,0.198838,0.719908
5,n5,n5,0.544941,0.220441,0.671413
6,n6,n6,0.809430,0.006499,0.844655
7,n7,n7,0.698139,0.340251,0.324384
8,n8,n8,0.957213,0.336595,0.274197
9,n9,n9,0.096716,0.847494,0.682981


In [6]:
pd.DataFrame(graph_data['edges'])

,id,source,target
0,e0,n7,n8
1,e1,n13,n10
2,e2,n10,n7
3,e3,n17,n9
4,e5,n15,n3
...,...,...,...
88,e95,n16,n7
89,e96,n18,n16
90,e97,n18,n13
91,e98,n19,n18


In [7]:
js_text_template = Template('''
    
    var g = $graph_data ;

s = new sigma({graph: g, container: '$container', settings: { defaultNodeColor: '#ec5148'} });

s.graph.nodes().forEach(function(n) {
  n.originalColor = n.color;
});
s.graph.edges().forEach(function(e) {
  e.originalColor = e.color;
});

s.bind('clickNode', function(e) {
  var nodeId = e.data.node.id,
      toKeep = s.graph.neighbors(nodeId);
  toKeep[nodeId] = e.data.node;

  s.graph.nodes().forEach(function(n) {
    if (toKeep[n.id])
      n.color = n.originalColor;
    else
      n.color = '#eee';
  });

  s.graph.edges().forEach(function(e) {
    if (toKeep[e.source] && toKeep[e.target])
      e.color = e.originalColor;
    else
      e.color = '#eee';
  });

  s.refresh();
});

s.bind('clickStage', function(e) {
  s.graph.nodes().forEach(function(n) {
    n.color = n.originalColor;
  });

  s.graph.edges().forEach(function(e) {
    e.color = e.originalColor;
  });

  s.refresh();
});


''')

In [8]:

js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})

In [9]:
html_template = Template('''
<div id="graph-div" style="height:800px"></div>
<script> $js_text </script>
''')

In [12]:
HTML(html_template.substitute({'js_text': js_text}))

In [11]:
class Point:
    """
	Exemple d'utilisation
		sommet = Point(x, y)
		id = sommet.getID()
		x, y = sommet.getCoords()		
	"""
    ID = 0 # Attribut UNIQUE d'identification de chaque point
    def __init__(self, x, y):
        # Init
        self.x = x
        self.y = y
        self.id = Point.ID
        Point.ID += 1
    def getID(self): return self.id
    def getCoords(): return self.x, self.y

class Chemin :
    # ok
    def __init__(self, pointInitial, pointFinal, poids):
        self.start = pointInitial
        self.final = pointFinal
        self.weight = poids
        
    def getChemin(self): 
        print(self.start.getID(), self.final.getID(), self.weight)
        return (self.start.getID(), self.final.getID(), self.weight)
        
    def getStart(self): return self.start

    def getFinal(self): return self.final

    def getWeight(self): return self.weight

# Classe du graphe
class Graph: 

	def __init__(self, vertices): 
		self.V = vertices # Number of vertices 
		self.graph = [] 
		self.parent = [None] * self.V

	# function to add an edge to graph 
	def addEdge(self, u, v, w): 
		self.graph.append([u, v, w]) 
	
	def addChemin(self, chemin): 
		self.graph.append(chemin.getChemin())
	# utility function used to print the solution 
	def printArr(self, dist): 
		print("\n Pondérations du point  SOURCE vers tous les autres POINTS") 
		for i in range(self.V): 
			print("{0}\t\t{1}".format(i, dist[i])) 
	
	# The main function that finds shortest distances from src to 
	# all other vertices using Bellman-Ford algorithm. The function 
	# also detects negative weight cycle 
	def BellmanFord(self, src): 

		# Step 1: Initialize distances from src to all other vertices 
		# as INFINITE 
		dist = [float("Inf")] * self.V 
		dist[src] = 0
		# parent = [None] * self.V

		# Step 2: Relax all edges |V| - 1 times. A simple shortest 
		# path from src to any other vertex can have at-most |V| - 1 
		# edges 
		for _ in range(self.V - 1): 
			# Update dist value and parent index of the adjacent vertices of 
			# the picked vertex. Consider only those vertices which are still in 
			# queue 
			for u, v, w in self.graph: 
				if dist[u] != float("Inf") and dist[u] + w < dist[v]: 
						dist[v] = dist[u] + w
						self.parent[v] = u

		# Step 3: check for negative-weight cycles. The above step 
		# guarantees shortest distances if graph doesn't contain 
		# negative weight cycle. If we get a shorter path, then there 
		# is a cycle. 

		for u, v, w in self.graph: 
				if dist[u] != float("Inf") and dist[u] + w < dist[v]: 
						print("Graph contains negative weight cycle") 
						return
						
		# print all distance 
		self.printArr(dist) 

	def PathPrinting(self, u) :
		v = self.parent[u]
		if v is None:
		    return
		self.PathPrinting(v)
		print(u)


g = Graph(6) 
# g.addEdge(0, 1, -1) 
# g.addEdge(0, 2, 4) 
# g.addEdge(1, 2, 3) 
# g.addEdge(1, 3, 2) 
# g.addEdge(1, 4, 2) 
# g.addEdge(3, 2, 5) 
# g.addEdge(3, 1, 1) 
# g.addEdge(4, 3, -3) 

p0 = Point(0, 5)
p1 = Point(10, -84)
p2 = Point(95, 4)
p3 = Point(54, -9)
p4 = Point(-2, 0)
p5 = Point(0, 0)

print('A - B - poids') 
g.addChemin(Chemin(p0, p1, -1)) 
g.addChemin(Chemin(p0, p2, 4)) 
g.addChemin(Chemin(p1, p2, 3)) 
g.addChemin(Chemin(p1, p3, 2)) 
g.addChemin(Chemin(p1, p4, 2)) 
g.addChemin(Chemin(p3, p2, 5)) 
g.addChemin(Chemin(p3, p1, 1)) 
g.addChemin(Chemin(p4, p3, -3)) 
g.addChemin(Chemin(p4, p5, 1)) 

# Print the solution
g.BellmanFord(0)

print('Points à passer pour aller à celui d\'ID ', 5)
g.PathPrinting(5)
# Initially, Contributed by Neelam Yadav 
# Later On, Edited by Himanshu Garg 

A - B - poids
0 1 -1
0 2 4
1 2 3
1 3 2
1 4 2
3 2 5
3 1 1
4 3 -3
4 5 1

 Pondérations du point  SOURCE vers tous les autres POINTS
0		0
1		-1
2		2
3		-2
4		1
5		2
Points à passer pour aller à celui d'ID  5
1
4
5
